# 下面是训练代码,训练过程中会进行测试.

In [1]:

import os
# import scipy.io.wavfile
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import sys
import numpy as np
from dnn_models import MLP, flip
from dnn_models import SincNet as CNN
from data_io import ReadList, read_conf, str_to_bool


In [19]:

#随机数据然后拼接成一个batch
def create_batches_rnd(batch_size, data_folder, wav_lst, N_snt, wlen, lab_dict, fact_amp):
    # Initialization of the minibatch (batch_size,[0=>x_t,1=>x_t+N,1=>random_samp])
    sig_batch = np.zeros([batch_size, wlen])
    lab_batch = np.zeros(batch_size)

    snt_id_arr = np.random.randint(N_snt, size=batch_size)

    rand_amp_arr = np.random.uniform(1.0 - fact_amp, 1 + fact_amp, batch_size)

    for i in range(batch_size):

        # select a random sentence from the list
        # [fs,signal]=scipy.io.wavfile.read(data_folder+wav_lst[snt_id_arr[i]])
        # signal=signal.astype(float)/32768

        [signal, fs] = sf.read(data_folder + wav_lst[snt_id_arr[i]])

        # accesing to a random chunk=======随机抽取一块语音
        snt_len = signal.shape[0]
        snt_beg = np.random.randint(snt_len - wlen - 1)  # randint(0, snt_len-2*wlen-1)
        snt_end = snt_beg + wlen

        channels = len(signal.shape)
        if channels == 2:
            print('WARNING: stereo to mono: ' + data_folder + wav_lst[snt_id_arr[i]])
            signal = signal[:, 0]
        #把语音乘以一个随机倍数然后拼接到空白语音上.
        sig_batch[i, :] = signal[snt_beg:snt_end] * rand_amp_arr[i]
        lab_batch[i] = lab_dict[wav_lst[snt_id_arr[i]]]

    inp = Variable(torch.from_numpy(sig_batch).float().contiguous())
    lab = Variable(torch.from_numpy(lab_batch).float().contiguous())

    return inp, lab

In [29]:
# 读取设置全部的超参数
import configparser as ConfigParser
from optparse import OptionParser
import numpy as np
# import scipy.io.wavfile
import torch
# Reading cfg file
if 1:
    class a():
      pass
    options=a()

    options.cfg = 'cfg/SincNet_TIMIT.cfg'
    cfg_file = options.cfg
    Config = ConfigParser.ConfigParser()
    Config.read(cfg_file)

    # [data]
    options.tr_lst = Config.get('data', 'tr_lst')
    options.te_lst = Config.get('data', 'te_lst')
    options.lab_dict = Config.get('data', 'lab_dict')
    options.data_folder = Config.get('data', 'data_folder')
    options.output_folder = Config.get('data', 'output_folder')
    options.pt_file = Config.get('data', 'pt_file')

    # [windowing]
    options.fs = Config.get('windowing', 'fs')
    options.cw_len = Config.get('windowing', 'cw_len')
    options.cw_shift = Config.get('windowing', 'cw_shift')

    # [cnn]
    options.cnn_N_filt = Config.get('cnn', 'cnn_N_filt')
    options.cnn_len_filt = Config.get('cnn', 'cnn_len_filt')
    options.cnn_max_pool_len = Config.get('cnn', 'cnn_max_pool_len')
    options.cnn_use_laynorm_inp = Config.get('cnn', 'cnn_use_laynorm_inp')
    options.cnn_use_batchnorm_inp = Config.get('cnn', 'cnn_use_batchnorm_inp')
    options.cnn_use_laynorm = Config.get('cnn', 'cnn_use_laynorm')
    options.cnn_use_batchnorm = Config.get('cnn', 'cnn_use_batchnorm')
    options.cnn_act = Config.get('cnn', 'cnn_act')
    options.cnn_drop = Config.get('cnn', 'cnn_drop')

    # [dnn]
    options.fc_lay = Config.get('dnn', 'fc_lay')
    options.fc_drop = Config.get('dnn', 'fc_drop')
    options.fc_use_laynorm_inp = Config.get('dnn', 'fc_use_laynorm_inp')
    options.fc_use_batchnorm_inp = Config.get('dnn', 'fc_use_batchnorm_inp')
    options.fc_use_batchnorm = Config.get('dnn', 'fc_use_batchnorm')
    options.fc_use_laynorm = Config.get('dnn', 'fc_use_laynorm')
    options.fc_act = Config.get('dnn', 'fc_act')

    # [class]
    options.class_lay = Config.get('class', 'class_lay')
    options.class_drop = Config.get('class', 'class_drop')
    options.class_use_laynorm_inp = Config.get('class', 'class_use_laynorm_inp')
    options.class_use_batchnorm_inp = Config.get('class', 'class_use_batchnorm_inp')
    options.class_use_batchnorm = Config.get('class', 'class_use_batchnorm')
    options.class_use_laynorm = Config.get('class', 'class_use_laynorm')
    options.class_act = Config.get('class', 'class_act')

    # [optimization]
    options.lr = Config.get('optimization', 'lr')
    options.batch_size = Config.get('optimization', 'batch_size')
    options.N_epochs = Config.get('optimization', 'N_epochs')
    options.N_batches = Config.get('optimization', 'N_batches')
    options.N_eval_epoch = Config.get('optimization', 'N_eval_epoch')
    options.seed = Config.get('optimization', 'seed')
print(options)





# [data]
tr_lst = options.tr_lst
te_lst = options.te_lst
pt_file = options.pt_file
class_dict_file = options.lab_dict
data_folder = options.data_folder + '/'
output_folder = options.output_folder

# [windowing]
fs = int(options.fs)
cw_len = int(options.cw_len)
cw_shift = int(options.cw_shift)

# [cnn]
cnn_N_filt = list(map(int, options.cnn_N_filt.split(',')))
cnn_len_filt = list(map(int, options.cnn_len_filt.split(',')))
cnn_max_pool_len = list(map(int, options.cnn_max_pool_len.split(',')))
cnn_use_laynorm_inp = str_to_bool(options.cnn_use_laynorm_inp)
cnn_use_batchnorm_inp = str_to_bool(options.cnn_use_batchnorm_inp)
cnn_use_laynorm = list(map(str_to_bool, options.cnn_use_laynorm.split(',')))
cnn_use_batchnorm = list(map(str_to_bool, options.cnn_use_batchnorm.split(',')))
cnn_act = list(map(str, options.cnn_act.split(',')))
cnn_drop = list(map(float, options.cnn_drop.split(',')))

# [dnn]
fc_lay = list(map(int, options.fc_lay.split(',')))
fc_drop = list(map(float, options.fc_drop.split(',')))
fc_use_laynorm_inp = str_to_bool(options.fc_use_laynorm_inp)
fc_use_batchnorm_inp = str_to_bool(options.fc_use_batchnorm_inp)
fc_use_batchnorm = list(map(str_to_bool, options.fc_use_batchnorm.split(',')))
fc_use_laynorm = list(map(str_to_bool, options.fc_use_laynorm.split(',')))
fc_act = list(map(str, options.fc_act.split(',')))

# [class]
class_lay = list(map(int, options.class_lay.split(',')))
class_drop = list(map(float, options.class_drop.split(',')))
class_use_laynorm_inp = str_to_bool(options.class_use_laynorm_inp)
class_use_batchnorm_inp = str_to_bool(options.class_use_batchnorm_inp)
class_use_batchnorm = list(map(str_to_bool, options.class_use_batchnorm.split(',')))
class_use_laynorm = list(map(str_to_bool, options.class_use_laynorm.split(',')))
class_act = list(map(str, options.class_act.split(',')))

# [optimization]
lr = float(options.lr)
batch_size = int(options.batch_size)
N_epochs = int(options.N_epochs)
N_batches = int(options.N_batches)
N_eval_epoch = int(options.N_eval_epoch)
seed = int(options.seed)

# training list
wav_lst_tr = ReadList(tr_lst)
wav_lst_tr = [i.upper() for i in wav_lst_tr]
snt_tr = len(wav_lst_tr)

# test list
wav_lst_te = ReadList(te_lst)
wav_lst_te = [i.upper() for i in wav_lst_te]
snt_te = len(wav_lst_te)
print(snt_te)


# Folder creation
try:
    os.stat(output_folder)
except:
    os.mkdir(output_folder)

# setting seed
torch.manual_seed(seed)
np.random.seed(seed)

# loss function
cost = nn.NLLLoss()

# Converting context and shift in samples
wlen = int(fs * cw_len / 1000.00)  # 取多长的序列
wshift = int(fs * cw_shift / 1000.00)

# Batch_dev
Batch_dev = 128

# Feature extractor CNN
CNN_arch = {'input_dim': wlen,
            'fs': fs,
            'cnn_N_filt': cnn_N_filt,
            'cnn_len_filt': cnn_len_filt,
            'cnn_max_pool_len': cnn_max_pool_len,
            'cnn_use_laynorm_inp': cnn_use_laynorm_inp,
            'cnn_use_batchnorm_inp': cnn_use_batchnorm_inp,
            'cnn_use_laynorm': cnn_use_laynorm,
            'cnn_use_batchnorm': cnn_use_batchnorm,
            'cnn_act': cnn_act,
            'cnn_drop': cnn_drop,
            }

1386


In [30]:
# 搭建网络结构

CNN_net = CNN(CNN_arch)
# CNN_net.cuda()

# Loading label dictionary
lab_dict = np.load(class_dict_file, allow_pickle=True).item()
lab_dict2 = {}
for i in lab_dict:
    lab_dict2[i.upper()] = lab_dict[i]
lab_dict = lab_dict2

# 这里面标签的含义是, 按照人来标的. 总共3696个语音.400多个人. 每一个人用一个int表示.从0--461
with open('for_debug', 'w') as f:
    f.write(str(lab_dict))

DNN1_arch = {'input_dim': CNN_net.out_dim,
             'fc_lay': fc_lay,
             'fc_drop': fc_drop,
             'fc_use_batchnorm': fc_use_batchnorm,
             'fc_use_laynorm': fc_use_laynorm,
             'fc_use_laynorm_inp': fc_use_laynorm_inp,
             'fc_use_batchnorm_inp': fc_use_batchnorm_inp,
             'fc_act': fc_act,
             }

DNN1_net = MLP(DNN1_arch)
# DNN1_net.cuda()


DNN2_arch = {'input_dim': fc_lay[-1],
             'fc_lay': class_lay,
             'fc_drop': class_drop,
             'fc_use_batchnorm': class_use_batchnorm,
             'fc_use_laynorm': class_use_laynorm,
             'fc_use_laynorm_inp': class_use_laynorm_inp,
             'fc_use_batchnorm_inp': class_use_batchnorm_inp,
             'fc_act': class_act,
             }

DNN2_net = MLP(DNN2_arch)

In [37]:
# 进行训练,打印每一个epoch的 err率
#=============接着训练.......直接读取之前训练好的权重做finetune即可.
print('加载上次训练完的模型')
try:
    pt_file=output_folder + '/model_raw.pkl'
    pt_file='exp/SincNet_TIMIT/model_raw.pkl'
    print('加载模型的路径是',pt_file)
    
    if pt_file != 'none':
        checkpoint_load = torch.load(pt_file)
        CNN_net.load_state_dict(checkpoint_load['CNN_model_par'])
        DNN1_net.load_state_dict(checkpoint_load['DNN1_model_par'])
        DNN2_net.load_state_dict(checkpoint_load['DNN2_model_par'])
except:
    print('加载失败')
    pass

optimizer_CNN = optim.RMSprop(CNN_net.parameters(), lr=lr, alpha=0.95, eps=1e-8)
optimizer_DNN1 = optim.RMSprop(DNN1_net.parameters(), lr=lr, alpha=0.95, eps=1e-8)
optimizer_DNN2 = optim.RMSprop(DNN2_net.parameters(), lr=lr, alpha=0.95, eps=1e-8)

for epoch in range(N_epochs):

    test_flag = 0
    CNN_net.train()
    DNN1_net.train()
    DNN2_net.train()

    loss_sum = 0
    err_sum = 0

    for i in range(N_batches):
        [inp, lab] = create_batches_rnd(batch_size, data_folder, wav_lst_tr, snt_tr, wlen, lab_dict, 0.2)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # print('shiyongdeshi', device)
        CNN_net = CNN_net.to(device)
        DNN1_net = DNN1_net.to(device)
        DNN2_net = DNN2_net.to(device)
        inp = inp.to(device)
        lab = lab.to(device)

        pout = DNN2_net(DNN1_net(CNN_net(inp)))  # 这里面就是自定义化的cnn 叫sinconv

        pred = torch.max(pout, dim=1)[1]
        loss = cost(pout, lab.long())
        err = torch.mean((pred != lab.long()).float())
        if i%1==0:
             print("当前epoch的loss和err", loss, err)

        optimizer_CNN.zero_grad()
        optimizer_DNN1.zero_grad()
        optimizer_DNN2.zero_grad()

        loss.backward()
        optimizer_CNN.step()
        optimizer_DNN1.step()
        optimizer_DNN2.step()

        loss_sum = loss_sum + loss.detach()
        err_sum = err_sum + err.detach()

    loss_tot = loss_sum / N_batches
    err_tot = err_sum / N_batches
    print('训练完一个epoch,保存模型')
    checkpoint = {'CNN_model_par': CNN_net.state_dict(),
                  'DNN1_model_par': DNN1_net.state_dict(),
                  'DNN2_model_par': DNN2_net.state_dict(),
                  }
    torch.save(checkpoint, output_folder + '/model_raw.pkl')


加载上次训练完的模型
加载模型的路径是 exp/SincNet_TIMIT/model_raw.pkl
加载失败


加载上次训练完的模型========GPU上运行结果:
当前epoch的loss和err tensor(0.0074, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0027, device='cuda:0')
当前epoch的loss和err tensor(0.0138, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0009, device='cuda:0')
当前epoch的loss和err tensor(0.0069, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0036, device='cuda:0')
当前epoch的loss和err tensor(0.0068, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0018, device='cuda:0')
当前epoch的loss和err tensor(0.0120, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0027, device='cuda:0')
当前epoch的loss和err tensor(0.0169, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0036, device='cuda:0')
当前epoch的loss和err tensor(0.0134, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0027, device='cuda:0')
当前epoch的loss和err tensor(0.0206, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0036, device='cuda:0')
当前epoch的loss和err tensor(0.0058, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0009, device='cuda:0')

看到最终准确率百分之99.8左右比传统的cnn结构更好